This notebook was inpired by [this LlamaIndex notebook](https://colab.research.google.com/drive/1c5ORIlqs3YMWosDSMgs6_ZHb5eiANS1c?usp=sharing)

Making some changes to it with the only intention of trying ideas and learning.

Notice that I am assuming you have the relevant API_KEYs as environmental variables.

In [2]:
from bubls.utils.agents.booking_tools import (
    get_booking_state_tool,
    update_booking_tool,
    create_booking_tool,
    confirm_booking_tool
)
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

Things to know:
- We will define an Agent that manages bookings for a restaurant.
    - We will use a `Booking` Pydantic class to track and represent a booking.
    - We will have the next methods:
        - `get_booking_state`: Use booking ID to retrieve the state of a booking. With return_direct=True, no LLM is used to generate the output
        - `update_booking`: Update the booking by modifying a specific property. With return_direct=False, LLM is used to generate the output
        - `create_booking`: Initialize a booking and requesting more info. With return_direct=True, no LLM is used to generate the output
        - `confirm_booking`: Complete the booking by making sure all information has been provided With return_direct=True, no LLM is used to generate the output

## Booking Tools

In [3]:
tools = [
    get_booking_state_tool,
    update_booking_tool,
    create_booking_tool,
    confirm_booking_tool,
]

In [4]:
prefix_messages = [
    ChatMessage(
        role="system",
        content=(
            f"You are now connected to the booking system and helping a user with making a booking. "
            "Only enter details that the user has explicitly provided."
            "Do not make up any details."
        ),
    )
]

In [5]:
worker = FunctionCallingAgentWorker(
    tools=tools,
    llm=OpenAI(model='gpt-3.5-turbo'),
    prefix_messages=prefix_messages,
    max_function_calls=10,
    allow_parallel_tool_calls=False,
    verbose=True,
)
agent = AgentRunner(worker)

In [18]:
response = agent.chat("Hello! I would like to make a booking.")

Added user message to memory: Hello! I would like to make a booking.
=== LLM Response ===
Great! To proceed with the booking, could you please provide me with your name, email, phone number, date, and time for the booking?


In [19]:
response = agent.chat("Sure! My name is bubl-ai, and my email is contact.bubl.ai@gmail.com")

Added user message to memory: Sure! My name is bubl-ai, and my email is contact.bubl.ai@gmail.com
=== Calling Function ===
Calling function: create_booking with args: {}
=== Function Output ===
Booking created with id KJKNYO, but not yet confirmed. Please provide your name, email, phone, date, and time.


In [20]:
response = agent.chat("My phone number is 111-222-3456, reservation for this sunday at 10 am")

Added user message to memory: My phone number is 111-222-3456, reservation for this sunday at 10 am
=== Calling Function ===
Calling function: update_booking with args: {"user_id": "KJKNYO", "property": "name", "value": "bubl-ai"}
=== Function Output ===
Booking ID KJKNYO updated with name = bubl-ai
=== Calling Function ===
Calling function: update_booking with args: {"user_id": "KJKNYO", "property": "email", "value": "contact.bubl.ai@gmail.com"}
=== Function Output ===
Booking ID KJKNYO updated with email = contact.bubl.ai@gmail.com
=== Calling Function ===
Calling function: update_booking with args: {"user_id": "KJKNYO", "property": "phone", "value": "111-222-3456"}
=== Function Output ===
Booking ID KJKNYO updated with phone = 111-222-3456
=== Calling Function ===
Calling function: update_booking with args: {"user_id": "KJKNYO", "property": "date", "value": "Sunday"}
=== Function Output ===
Booking ID KJKNYO updated with date = Sunday
=== Calling Function ===
Calling function: updat

In [21]:
response = agent.chat("yes")

Added user message to memory: yes


=== Calling Function ===
Calling function: confirm_booking with args: {"user_id": "KJKNYO"}
=== Function Output ===
Booking for Name: bubl-ai and ID: KJKNYO is confirmed!


In [22]:
response = agent.chat("provide the booking details")

Added user message to memory: provide the booking details
=== Calling Function ===
Calling function: get_booking_state with args: {"user_id": "KJKNYO"}
=== Function Output ===
{'name': 'bubl-ai', 'email': 'contact.bubl.ai@gmail.com', 'phone': '111-222-3456', 'date': 'Sunday', 'time': '10:00 AM'}


In [23]:
response = agent.chat("List all user the IDs that exist and the name is associated with each.")

Added user message to memory: List all user the IDs that exist and the name is associated with each.


=== LLM Response ===
Here is the list of user IDs and the associated names:

1. User ID: KJKNYO, Name: bubl-ai
